In [0]:
%tensorflow_version 1.x

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from google.colab.patches import cv2_imshow
import os

# List of keras tools
from keras import optimizers
from keras.models import Model
from keras import applications
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [0]:
!wget 'https://fyp-content-bucket.s3.amazonaws.com/Dataset.zip'

--2019-11-11 19:54:48--  https://fyp-content-bucket.s3.amazonaws.com/Dataset.zip
Resolving fyp-content-bucket.s3.amazonaws.com (fyp-content-bucket.s3.amazonaws.com)... 52.216.133.251
Connecting to fyp-content-bucket.s3.amazonaws.com (fyp-content-bucket.s3.amazonaws.com)|52.216.133.251|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4013338444 (3.7G) [application/zip]
Saving to: ‘Dataset.zip’

Dataset.zip         100%[===================>]   3.74G  13.0MB/s    in 5m 5s   

2019-11-11 19:59:53 (12.6 MB/s) - ‘Dataset.zip’ saved [4013338444/4013338444]



In [0]:
!unzip Dataset.zip

Archive:  Dataset.zip
   creating: Dataset/
   creating: Dataset/MICC-F2000/
   creating: Dataset/MICC-F2000/Test/
   creating: Dataset/MICC-F2000/Train/
   creating: Dataset/MICC-F2000/Test/Tampered/
   creating: Dataset/MICC-F2000/Test/Authentic/
   creating: Dataset/MICC-F2000/Train/Tampered/
   creating: Dataset/MICC-F2000/Train/Authentic/
  inflating: Dataset/MICC-F2000/Test/Tampered/P7290459tamp11.jpg  
  inflating: Dataset/MICC-F2000/Test/Tampered/P1000231tamp11.jpg  
  inflating: Dataset/MICC-F2000/Test/Tampered/DSC01172tamp13.jpg  
  inflating: Dataset/MICC-F2000/Test/Tampered/P4120010tamp8.jpg  
  inflating: Dataset/MICC-F2000/Test/Tampered/sony_61tamp7.jpg  
  inflating: Dataset/MICC-F2000/Test/Tampered/P7310182tamp5.jpg  
  inflating: Dataset/MICC-F2000/Test/Tampered/P7310182tamp4.jpg  
  inflating: Dataset/MICC-F2000/Test/Tampered/DSC_1716tamp11.jpg  
  inflating: Dataset/MICC-F2000/Test/Tampered/DSCF18tamp6.jpg  
  inflating: Dataset/MICC-F2000/Test/Tampered/DSC_0254tamp9

In [0]:
batch_size = 67
train_samples = 1340
validation_samples = 660

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    'Dataset/MICC-F2000/Train',                  
    target_size=(224, 224),        
    batch_size=batch_size,     
    class_mode='binary',           
    color_mode='rgb',
    shuffle = True,
    seed= 42)           

validation_generator = test_datagen.flow_from_directory(
    'Dataset/MICC-F2000/Test',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode="binary",
    color_mode="rgb",    
    shuffle=True,
    seed=42)


Found 1340 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [0]:
# BUILD THE MODEL
def create_model():
    data_format="channels_first"

    model = Sequential()

    model.add(Convolution2D(32, (3, 3), input_shape=(224, 224, 3)))
    model.add(Activation('relu'))
   
    model.add(MaxPooling2D(data_format=data_format, pool_size=(2, 2)))

    model.add(Convolution2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(data_format=data_format, pool_size=(2, 2)))

    model.add(Convolution2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(data_format=data_format, pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(optimizer='rmsprop',           
                  loss='binary_crossentropy',    
                  metrics=['accuracy'])
    
    return model

In [0]:
# Train the network. Note that this may take a few minutes
model = create_model()
model.fit_generator(
        train_generator,
        steps_per_epoch=train_samples // batch_size,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=validation_samples // batch_size)

Epoch 1/20
20/20 [==============================] - 135s 7s/step - loss: 5.3876 - acc: 0.6410 - val_loss: 5.6400 - val_acc: 0.6501
Epoch 2/20
20/20 [==============================] - 164s 8s/step - loss: 5.6413 - acc: 0.6500 - val_loss: 5.5177 - val_acc: 0.6577
Epoch 3/20
20/20 [==============================] - 126s 6s/step - loss: 5.6413 - acc: 0.6500 - val_loss: 5.5177 - val_acc: 0.6577
Epoch 4/20
20/20 [==============================] - 126s 6s/step - loss: 5.6413 - acc: 0.6500 - val_loss: 5.8166 - val_acc: 0.6391
Epoch 5/20
20/20 [==============================] - 126s 6s/step - loss: 5.2928 - acc: 0.6321 - val_loss: 0.6879 - val_acc: 0.6341
Epoch 6/20
20/20 [==============================] - 127s 6s/step - loss: 0.8332 - acc: 0.6119 - val_loss: 0.6796 - val_acc: 0.6290
Epoch 7/20
20/20 [==============================] - 127s 6s/step - loss: 0.6657 - acc: 0.6291 - val_loss: 0.6477 - val_acc: 0.6830
Epoch 8/20
20/20 [==============================] - 126s 6s/step - loss: 0.7300 - a

In [0]:
# Save the model and the weights on the drive.
model.save_weights("first_try.h5")

In [0]:
WEIGHTS_FILE = 'first_try.h5'

model = create_model()
model.load_weights(WEIGHTS_FILE)

# Load Real images into an array
REAL_IMAGES = ['Dataset/MICC-F2000/Train/Authentic/DCAM0158_scale.jpg',
               'Dataset/MICC-F2000/Train/Authentic/DSC_0021_01_scale.jpg',
               'Dataset/MICC-F2000/Train/Authentic/DSC_0261_01_scale.jpg']
                
# Load Fake images into an array
FAKE_IMAGES = ['Dataset/MICC-F2000/Train/Tampered/DSCF18tamp1.jpg',
               'Dataset/MICC-F2000/Train/Tampered/DSCN41tamp14.jpg',
               'Dataset/MICC-F2000/Train/Tampered/DSC_0648tamp12.jpg']

for i in REAL_IMAGES:
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def test_model(array_of_images, model):
    # Loop through the array of images
    for i in range(0, len(array_of_images)):
        # Load the image and resize it to the network input layer (150, 150)
        img = load_img(array_of_images[i], target_size=(224,224))
        
        # Remove the plot x and y ticks
        plt.xticks([])
        plt.yticks([])
        
        # Show the image
        plt.imshow(img)
        plt.show()
        
        # Do transformations on the image so that it can be input as an argument to
        # the model prediction
        img = img_to_array(img)
        img = img.reshape((1,) + img.shape)
        img_classes = model.predict_classes(img)
        
        if img_classes[0] == 0:
            print("The image above is a REAL!")
        else:
            print("The image above is a FAKE!")

test_model(REAL_IMAGES, model)
test_model(FAKE_IMAGES, model)

NameError: ignored